<a href="https://colab.research.google.com/github/vivekkishore/machinelearning/blob/main/querygpt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Attempt to query excel using unstructured excel loader

In [ ]:
!pip install python-dotenv
!pip install openai
!pip install --upgrade langchain
!pip install tiktoken
!pip install chromadb
!pip install unstructured
import warnings
warnings.filterwarnings('ignore')
import langchain
langchain.debug=False
import os, openai
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())
openai.api_key=os.environ['API_KEY']

In [15]:
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredExcelLoader
#Load
loader = UnstructuredExcelLoader("/content/Sample2.xlsx")
docs = loader.load()

# # Split
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0,separators=["\n\n\n"])
texts=text_splitter.split_documents(docs)


In [ ]:
# docs
# texts

In [30]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


from langchain.vectorstores import Chroma
persist_directory = '/content/chroma/'

# to remove old database files
# !rm -rf /content/chroma/

db=Chroma.from_documents(documents=docs, embedding=embedding, persist_directory=persist_directory)


In [34]:
#fetching relevant data directly from vector db using similarilty search
# Query='What is the NameID for Pluralsight vendor'
# result=db.similarity_search(Query, k=2)
# result1=db.max_marginal_relevance_search(Query,fetch_k=4,k=3)

# print(result1)

from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0,openai_api_key= openai.api_key)

# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

# Run chain
from langchain.chains import RetrievalQA

def run_model(question):
  qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type='mmr',search_kwargs={'k': 3, 'fetch_k':4 }),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}, verbose =True)
  result=qa_chain({"query":question})
  return result

[Document(page_content="\n\n\nAIR ID\nVendor Name\nApplication Status\nIntegration Specific\nIntegration Approach\nSP/IDP\nIntermediate screen present to enter UPN?\nAD Group Restriction\nSupported by LVA Team\n(Yes/No)\nRoles\nSupports Separate Stage & Production Environments\nSuperGroup\nYes/No\nGroup name vs Object ID\nStage AD Group/Stage SuperGroup Name\nProduction AD Group/Production SuperGroup Name\nProduction AD Group Owner/s\nAuto Provisioning\nName ID\nIf Name ID is not People Key. What is it & why?\nName ID format\nSupport for Prejoiners\nSupport for Contractors\nSupport for V&A\nMobile SSO\nmyL Asset - Yes/No\nIf Q= Yes, then Course Information\nADFS Relying Parties/Azure SSO Decommossioned\nDecommission RITMs\nVendor Contact\nBusiness Owner Contact\nVendor Server Hosting Location Country\nStage Cert Exp\nProd cert Exp\nStage Active / Inactive\n\n\n\n\n10040\nPluralsight\nActive\nMigrated to Azure AD\nSP\nNo\nYes, multiple AD groups inside the SG to put users in specific le

In [ ]:
 run_model('What is the NameID for Pluralsight vendor')

In [ ]:
# including memory to store past conversation
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
from langchain.chains import ConversationalRetrievalChain

def run_memory_chain(question):
  qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=db.as_retriever(search_type='similarity',search_kwargs={'k': 3, 'fetch_k':4 }),
    chain_type="stuff",
    memory=memory,
    verbose=True)

  result =qa({"question":question})
  return result



In [ ]:
run_memory_chain('What is the NameID for Pluralsight vendor?')

###using csvloader

In [ ]:
!pip install python-dotenv
!pip install openai
!pip install --upgrade langchain
!pip install tiktoken
!pip install chromadb
import warnings
warnings.filterwarnings('ignore')
import langchain
import os, openai
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())
openai.api_key=os.environ['API_KEY']

###Load but no split

In [38]:
# import pandas as pd
# pd1=pd.read_excel('Sample2.xlsx')
# pd1.to_csv('Sample3.csv', index=False)

from langchain.document_loaders import CSVLoader

loader=CSVLoader('Sample3.csv')

docs=loader.load()

In [40]:
len(docs)

52

###putting in chromadb and quering

In [ ]:
# docs
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)

#doc itself is already splitted as pr number of rows and have context attached according to the excel headers.

from langchain.vectorstores import Chroma
persist_directory = '/content/chroma/'

db=Chroma.from_documents(documents=docs, embedding=embedding, persist_directory=persist_directory)

# to remove old database files
# !rm -rf /content/chroma/

#fetching relevant data directly from vector db using similarilty search
# Query='What is the NameID for Pluralsight vendor'
# result=db.similarity_search(Query, k=2)
# result1=db.max_marginal_relevance_search(Query,fetch_k=4,k=3)

# result1


### query using LLM

In [44]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0,openai_api_key= openai.api_key)

# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

# Run chain
from langchain.chains import RetrievalQA

def run_model(question):
  qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type='mmr',search_kwargs={'k': 3, 'fetch_k':4 }),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}, verbose =True)
  result=qa_chain({"query":question})
  return result

In [45]:
run_model('what is nameid for pluralsight?')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what is nameid for pluralsight?',
 'result': 'The name ID for Pluralsight is PeopleKey.',
 'source_documents': [Document(page_content='AIR ID: 10040\nVendor Name: Pluralsight\nApplication Status: Active\nIntegration Specific: Migrated to Azure AD\nIntegration Approach\nSP/IDP: SP\nIntermediate screen present to enter UPN?: No\nAD Group Restriction: Yes, multiple AD groups inside the SG to put users in specific learning paths.\nSupported by LVA Team\n(Yes/No): Yes\nRoles: User, Team Manager, Admin\nSupports Separate Stage & Production Environments: Yes\nSuperGroup\nYes/No: Yes\nGroup name vs Object ID: Object IDs are being passed under groups claim in SAML.\nStage AD Group/Stage SuperGroup Name: Pluralsight_SSO_SG\nProduction AD Group/Production SuperGroup Name: Pluralsight_SSO_SG\nProduction AD Group Owner/s: corina.dobos\noana.florica\ndijosh.thannikkattil\nAuto Provisioning: Yes\nName ID: PeopleKey\nIf Name ID is not People Key. What is it & why?: Not available\nName ID fo

###query with memory buffer

In [46]:
# including memory to store past conversation
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
from langchain.chains import ConversationalRetrievalChain

def run_memory_chain(question):
  qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=db.as_retriever(search_type='similarity',search_kwargs={'k': 3, 'fetch_k':4 }),
    chain_type="stuff",
    memory=memory,
    verbose=True)

  result =qa({"question":question})
  return result



In [47]:
run_memory_chain('what is nameid for pluralsight?')



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
AIR ID: 10040
Vendor Name: Pluralsight
Application Status: Active
Integration Specific: Migrated to Azure AD
Integration Approach
SP/IDP: SP
Intermediate screen present to enter UPN?: No
AD Group Restriction: Yes, multiple AD groups inside the SG to put users in specific learning paths.
Supported by LVA Team
(Yes/No): Yes
Roles: User, Team Manager, Admin
Supports Separate Stage & Production Environments: Yes
SuperGroup
Yes/No: Yes
Group name vs Object ID: Object IDs are being passed under groups claim in SAML.
Stage AD Group/Stage SuperGroup Name: Pluralsight_SSO_SG
Production AD Group/Production SuperGroup Name: Pluralsight_SSO_SG
Production AD Group Owner/s: corina.dobos
oana.florica
dijosh.thannikk

{'question': 'what is nameid for pluralsight?',
 'chat_history': [HumanMessage(content='what is nameid for pluralsight?', additional_kwargs={}, example=False),
  AIMessage(content='The Name ID for Pluralsight is "PeopleKey".', additional_kwargs={}, example=False)],
 'answer': 'The Name ID for Pluralsight is "PeopleKey".'}

In [49]:
# memory.clear()
# memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history')